In [13]:
import pandas as pd
import urllib.request

# dataframe from API
def lc_pd_dataframe(url):
    livechart = "http://localhost:8080/relnsdtest/v1/data?"
    url = livechart + url
    req = urllib.request.Request(url)
    req.add_header('User-Agent', 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:77.0) Gecko/20100101 Firefox/77.0')
    return pd.read_csv(urllib.request.urlopen(req))

# nuclide id from Z,N,Element 
def nucid_from_ds(dataset):
    return str(dataset['d_z'].iloc[0] + dataset['d_n'].iloc[0]) + dataset['d_symbol'].iloc[0]
    

#
# Decay chain from ENSDF decay datasets. Considers ONLY decays for which there is an evaluated
# datatset ( --> decay radiations are detected)

# THE CODE ASSUMES THAT: 
#    1) THERE IS ONLY ONE DECAY BRANCH (plus a transiton to the GS in case of a metastable) 
#               This needs to be extended to multiple branches. It means the decay path does not split and/or rejoins
#    2) THE DECAYS CONSIDERED ARE B-, ALPHA, EC/B+ . Need to think about cases like n or p emission etc...
#    2) THE METASTABLE DECAYS to the GS, AND TO THE SAME NUCLIDE AS THE GS (e.g. in 99-Tc both GS and m go to 99-Ru)
#    3) THE DECAY SCHEMA IS WELL DESCRIBED. CASES LIKE Cd-131 DECAY (no beta- intensities, no gamma cascade) WOULD NOT WORK

# WARNING !!!!! I DID ONLY SOME TESTING !!!!!

# I would advise to check the decays on livechart, to understand what is going on. 

# let's go:

# set a threshold for metastable H-l, in seconds
meta_th_s =  (60*2) # try with 2 minutes 

# data structure to save the links of the chain 
class Decay_link():
    def __init__(self, parent, daughter ,decay_mode,branching, meta, processed):
        self.parent = parent
        self.daughter = daughter
        self.decay_mode = decay_mode
        self.branching = branching # normalised to 1 !
        self.meta = meta
        self.processed = processed
        
# instances of Decay_link
decay_links = []


# takes and returns an instance of Decay_link
def decay(link):
    
    # the link is processed
    link.processed = True
    
    par_id = link.daughter
    # get all radiations types from par_id decay
    df_bm = lc_pd_dataframe('fields=decay_rads&nuclides='+ par_id +'&rad_types=bm')
    df_al = lc_pd_dataframe('fields=decay_rads&nuclides='+ par_id +'&rad_types=a')
    df_bp = lc_pd_dataframe('fields=decay_rads&nuclides='+ par_id +'&rad_types=bp')
    df_gm = lc_pd_dataframe('fields=decay_rads&nuclides='+ par_id +'&rad_types=g')
    
    # whether there is any decay
    sum_dec = (not df_bm.empty) + (not df_al.empty) + (not df_bp.empty) 
    
    if(sum_dec == 0 and df_gm.empty):
        # no b+,b-,a,g : should be stable
        print('probably stable')
        return None
    
    if(sum_dec == 0 and not df_gm.empty): 
        # only gammas
        
        # check how many decay modes are there
        gb = df_gm.groupby('decay')
       
        keys = list(gb.groups.keys()) 
        groups = dict(list(gb))

        # only one decay mode, and it is Isomeric Transition
        if(len(keys) == 1 and keys[0] == 'IT'):
            print(' Only IT to GS, probably stable')
            return None

        # there is a gamma cascade from b-,a, or b+ decay, but no such radiations, probably not well measured decay. See for example Xe-116 decay
        print('probably incomplete decay description')
        for k in keys:
            id_dau = str(groups[k]['d_n'][0] + groups[k]['d_z'][0]) + groups[k]['d_symbol'][0]
            
        # the last decay. The case of more than one decay needs to be implemented
        decay_links.append(Decay_link(par_id,id_dau, k, groups[k]['decay_%'][0]/100, groups[k]['p_energy'][0]>0, False))
        return decay_links[-1]
    
    dec_perc = link.branching * 100
    if(sum_dec > 1):
        # more than one decay !
        # for the moment take the first
        print('    more than one decay !')
        if(not df_bp.empty): 
            dec_perc = df_bp['decay_%'].iloc[0] * link.branching
            print('    b+/EC ', dec_perc , ' to ', nucid_from_ds(df_ec), ' - per 100 decay of the parent : ', df_bp['decay_%'].iloc[0] )
        if(not df_al.empty): 
            dec_perc = df_al['decay_%'].iloc[0] * link.branching
            print('    alpha ', dec_perc , ' to ', nucid_from_ds(df_al),' - per 100 decay of the parent :', df_al['decay_%'].iloc[0] )
        if(not df_bm.empty): 
            dec_perc = df_bm['decay_%'].iloc[0] * link.branching
            print('    b- ', dec_perc , ' to ', nucid_from_ds(df_bm),' - per 100 decay of the parent : ', df_bm['decay_%'].iloc[0] )
        
        
        
    # take only one decay. The case of more than one decay needs to be implemented
    df = df_bm if not df_bm.empty else df_al if not df_al.empty else df_bp
    dec_type = 'bm' if not df_bm.empty else 'a' if not df_al.empty else 'bp'
 
    # extract, if any, the b+, or a, or b- lines feeding a metastable:
    # first condition: energy of the fed level is above 0
    df_meta = df[(df[['daughter_level_energy']] > 0).all(axis=1)]
    # second condition: H-l above the treshold
    df_meta = df_meta[(df_meta[['daughter_level_hl']] > meta_th_s).all(axis=1)] #caveat: in principle there could be more than one metastable !
    # replace NaN with 0
    df_meta = df_meta.fillna(0)
    
    # if df_meta is not empty, there is a mestable
    if(not df_meta.empty):
        # nuclide id of the meta
        id_meta = nucid_from_ds(df_meta) 

        # extract the H-l of the metastable (for Tc-99m is 21625.92 s)
        meta_hl = df_meta['daughter_level_hl'].iloc[0]
        # the intensity of the line feeding the mestable, normalised to 1
        if(dec_type == 'bm'):
            ratio_meta = df_meta['intensity_beta'].iloc[0] /100
        elif(dec_type == 'a'):
             ratio_meta = df_meta['intensity'].iloc[0] /100
        else:
            # b+/ec : sum the positron emission (b+) and the electron capture
            ratio_meta = (df_meta['intensity_beta'].iloc[0] + df_meta['intensity_ec'].iloc[0]) /100

        # To the intensity above one needs to add the gammas, from the cascade, that are populating the metastable. 
        # Please note that this ENSDF dataset contains the gammas that are emitted immediately after the beta, or alpha, decay.
        # These include the gammas that are populating the metastable, but NOT necessarely the gammas emitted by the 
        # meta stable level, which are placed in a different dataset: the one describing the metastable decay (see below df_meta_dec) 
        # Please check the Mo-99 decay on Livechart:
        # The table with gammas does not have the intensities for the ones emitted by the 140.511 keV level of 99-Tc. These can be seen 
        # by going to the 99-Tc decays, and selecting the IT decay
        
        # select the gammas populating the metastable
        df_g_meta = df_gm[( df_gm[['end_level_hl']] ==  meta_hl ).all(axis=1) ]  
        # caveat: it is not guaranteed the exact equality of half_lives! 
        # better, i think, is to allow a bit of margin like  abs(df_gm[['end_level_hl']] -  meta_hl) < 2, or might be a percentage

        # there might be cases in which no gammas go to the meta, check if there are any
        if (df_g_meta.groupby('end_level_hl').end_level_hl.nunique().iloc[0] > 0):

            # the mestable production is the direct beta or alpha feeding already calculated above, plus the gammas from levels above the meta.
            # the total gamma transition is:  photon_intensity * (1 + conversion_coefficient)
            ratio_meta = ratio_meta + (df_g_meta['intensity'] *(1+df_g_meta['conversion_coeff'].fillna(0))).sum()/100

        # Now we want to get how much of the metastable does NOT go to its GS, but to another nuclide (e.g Tc-99m goes to Tc-99gs and Ru-99):
        # Retrieve with the API the gammas emitted by, e.g., Tc-99 (remember, this dataset contains the gammas emitted by the gs and the meta), 
        # and get how much of the metastable does NOT decay via IT (isomeric transition).
        # The metastable is selected using the half_life field, then !IT excludes the isomeric transition to the GS. 
        # What is left is the decay(s) from the meta to a different nuclide, in the case of 99-Tc m, it is Ru-99
        # caveat, the metastable could in principle have more than one decay mode

        df_meta_dec = lc_pd_dataframe('fields=decay_rads&nuclides=' + id_meta + '&rad_types=g').query("decay!='IT'").query("half_life_sec == " + str(meta_hl))
        # same caveat as above for H-l matching
        id_dau = nucid_from_ds(df_meta_dec) 
        
        # for 100 decays of the meta, how many are not IT
        perc_meta = df_meta_dec['decay_%'].iloc[0]

        # perc_meta is for 100 decays of the meta, but now only ratio_meta are produced
        perc_gs = dec_perc - perc_meta * ratio_meta

        print(par_id + ' to ' + id_meta + ' GS [%]: ' , perc_gs, ' ( direct from b-/b+/alpha   + through the metastable )')
        print(par_id + ' to ' + id_meta + ' m  [%]: ' , ratio_meta)

        # the % for Mo-99 to 99-Ru, any path:
        print(par_id + ' to ' + id_dau + '[%], any path :', dec_perc) 
        print(par_id + ' to ' + id_dau + ' through ' + id_meta + ' m  [%]: ' , perc_meta * ratio_meta)
        
        decay_links.append(Decay_link(par_id,id_dau, dec_type, perc_meta/100, True, False))

        return decay_links[-1]
    # no metastable
    else:
        id_dau = nucid_from_ds(df) 
        # approximation of only 1 branching
        print(par_id +  ' ' + dec_type + ' to ' + id_dau + ' [%]', dec_perc)
        # save some info in case of further need
        decay_links.append(Decay_link(par_id,id_dau, dec_type, dec_perc/100, False, False))
        return decay_links[-1]
    

nucid = '99mo'    # b-
nucid = '235u'  # Mix of a and b-, when reaching Bi-211 there are two branchings: a and b- 
                  # Now takes only one branch, good case for testing, when multiple br will be implemented
#nucid = '116xe' # EC/b+
#nucid = '211bi' # for testing the extension to two branches
link = Decay_link(None,nucid,None,1,None,False)
decay_links.append(link)
while( link != None):
    print('process ' + link.daughter)
    link = decay(link)
    print ('')


process 235u
235u a to 231Th [%] 100

process 231Th
231Th bm to 231Pa [%] 100.0

process 231Pa
231Pa a to 227Ac [%] 100.0

process 227Ac
    more than one decay !
    alpha  1.38  to  223Fr  - per 100 decay of the parent : 1.38
    b-  98.62  to  227Th  - per 100 decay of the parent :  98.62
227Ac bm to 227Th [%] 98.62

process 227Th
227Th a to 223Ra [%] 98.62

process 223Ra
223Ra a to 219Rn [%] 98.62

process 219Rn
219Rn a to 215Po [%] 98.62

process 215Po
215Po a to 211Pb [%] 98.62

process 211Pb
211Pb bm to 211Bi [%] 98.62

process 211Bi
    more than one decay !
    alpha  98.34780880000001  to  207Tl  - per 100 decay of the parent : 99.724
    b-  0.2721912  to  211Po  - per 100 decay of the parent :  0.276
211Bi bm to 211Po [%] 0.2721912

process 211Po
211Po a to 207Pb [%] 0.2721912

process 207Pb
 Only IT to GS, probably stable

